In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,SimpleRNN,GRU

In [ ]:
with open("/content/WhatsApp Chat.txt", 'r') as file:
  data=file.read()

In [ ]:
data

'3/4/24, 22:03 - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n3/4/24, 22:03 - You created group "Discussion"\n3/4/24, 22:05 - Pranesh: Hi, We will discuss about emotion based any topic.\n3/4/24, 22:22 - Pranesh: I\'m happy to create the new discussion group.\n3/10/24, 01:01 - Pranesh: Let\'s start\n3/10/24, 01:01 - Pranesh: What about your study?\n3/10/24, 01:03 - Urahara: It is pretty good but I am feeling very nervous about my ongoing exam.\n3/10/24, 01:05 - Pranesh: Oh, It\'s not a big deal. I also faced this way.\n3/10/24, 01:06 - Urahara: Yeah it is common feeling in the time of exam. Everyone faced this problem at least one time in time of exam\n3/10/24, 01:07 - Pranesh: Don\'t panic too much\n3/10/24, 01:07 - Pranesh: Easy man.\n3/10/24, 01:07 - Urahara: Do you ever feel like this in your exam?\n3/10/24, 01:07 - Pranesh: Yes, Many times\n3/10/24, 01:09 - Urahara: Iam trying to overcom

In [ ]:
chat_history = []
for i in data.split('\n'):
  try:
   chat_history.append(i.split(': ')[1])
  except:
    pass

In [ ]:
tokenize = Tokenizer()
tokenize.fit_on_texts(chat_history)

In [ ]:
total_words = len(tokenize.word_index)+1

In [ ]:
total_words

251

In [ ]:
input_sequence=[]
for line in chat_history:
    token_list=tokenize.texts_to_sequences([line])[0]

    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequence.append(n_gram_sequence)

In [ ]:
# Pad Sequence
max_sequence_len=max([len(x) for x in input_sequence])
input_sequence=np.array(pad_sequences(input_sequence,maxlen=max_sequence_len,padding='pre'))
input_sequence

array([[  0,   0,   0, ...,   0, 110,  60],
       [  0,   0,   0, ..., 110,  60,  17],
       [  0,   0,   0, ...,  60,  17, 111],
       ...,
       [  0,   0,   0, ...,  24,  44,  13],
       [  0,   0,   0, ...,  44,  13,  23],
       [  0,   0,   0, ...,  13,  23, 250]], dtype=int32)

In [ ]:
input_sequence.shape

(599, 27)

In [ ]:
x = input_sequence[:,:-1]
y = input_sequence[:,-1]

In [ ]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
y.shape

(599, 251)

In [ ]:
y[30]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 16,))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(x,y,epochs=100)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.0124 - loss: 5.5162
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.0169 - loss: 5.2069
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.0527 - loss: 5.0240
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0547 - loss: 5.0028
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.0441 - loss: 4.9908
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0470 - loss: 4.9749
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.0556 - loss: 4.9707
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.0420 - loss: 4.9405
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.0549 - loss: 4.8191
Epoch 10/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.0486 - loss: 4.8251
Epoch 11/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.0600 - loss: 4.8035
Epoch 12/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step

In [ ]:
# Prediction
def predict_next_word(seed_text,num_words=2):
    for _ in range(num_words):
        token_list=tokenize.texts_to_sequences([seed_text])[0]

        token_list_padded=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')

        predicted=np.argmax(model.predict(token_list_padded,verbose=0),axis=-1)

        for word,index in tokenize.word_index.items():
            if index==predicted:
                seed_text+=' '+word
                break
    return seed_text

In [ ]:
predict_next_word('hello')

'hello you cool'

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 26, 16)         │         4,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │        74,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 251)            │        32,379 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 331,907 (1.27 MB)

 Trainable params: 110,635 (432.17 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 221,272 (864.35 KB)